# Implementing Photon-Beetle-Hash Function

# Importing Necessary Libraries

In [1]:
import random
import copy
from collections import deque
import galois
GF = galois.GF(2**4)
print(GF.properties)

Galois Field:
  name: GF(2^4)
  characteristic: 2
  degree: 4
  order: 16
  irreducible_poly: x^4 + x + 1
  is_primitive_poly: True
  primitive_element: x


# PHOTON<sub>256</sub>-Permutation Function

In [2]:
# ---------------------------------------- PHOTON-256-PERMUTATION ---------------------------------------------
## SBOX LIST
sbox_list = [0xc,5,6,0xb,9,0,0xa,0xd,3,0xe,0xf,8,4,7,1,2]
def list_64_to_8x8_matrix(s) -> list[list[int]]:
    assert len(s) == 64
    m = [[s[i+(j*8)] for j in range(8)] for i in range(8)]
    return m
def matrix_8x8_to_hex_list(m : list[list[int]]):
    lst = []
    for col in range(8):
        for row in range(8):
            lst.append(m[row][col])
    return lst
## 1.ADD-CONSTANT
def add_constant(X : list,k : list):
    new_X = copy.deepcopy(X)
    RC = [1, 3, 7, 14, 13, 11, 6, 12, 9, 2, 5, 10]
    IC = [0, 1, 3, 7, 15, 14, 12, 8]
    for i in range(8):
        new_X[i][0] = new_X[i][0] ^ RC[k] ^ IC[i]
    return new_X
## 2.SUB-CELL
def sub_cell(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        for j in range(8):
            new_X[i][j] = sbox_list[new_X[i][j]]
    return new_X
## 3.SHIFT-ROW
def shift_row(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        temp = deque(new_X[i])
        temp.rotate(-1*i)
        new_X[i] = list(temp)
    return new_X
## 4.MIX-COLUMN-SERIAL
def serial(lst):
    M = []
    for i in range(7):
        a = [0 for j in range(8)]
        a[i+1] = 1
        M.append(a)
    M.append(copy.deepcopy(lst))
    return M
def matrix_mul(m1,m2):
    new_m = [[0 for j in range(8)] for i in range(8)]
    for i in range(8):
        for j in range(8):
            s = 0
            for temp in range(8):
                s ^= int(GF(m1[i][temp]) * GF(m2[temp][j]))
            new_m[i][j] = s
    return new_m
def mix_column_serial(X):
    new_X = copy.deepcopy(X)
    M = serial([2, 4, 2, 11, 2, 8, 5, 6])
    M8 = matrix_mul(M,M)
    for i in range(6):
        M8 = matrix_mul(M8,M)
    new_X = matrix_mul(M8,new_X)
    return new_X
## PHOTON 256 PERMUTATION FUNCTION
def PHOTON_256(input_hex_str = "0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef"):
    if type(input_hex_str) == "":
        input_hex_str = [int(i,16) for i in input_hex_str]
    assert len(input_hex_str) == 64
    X = list_64_to_8x8_matrix(input_hex_str)
    # 0 to 11
    for i in range(12):
        X = add_constant(X,i)
        X = sub_cell(X)
        X = shift_row(X)
        X = mix_column_serial(X)
    X = matrix_8x8_to_hex_list(X)
    assert len(X) == 64
    return X

# Some Utility Functions

In [3]:
def divide_chunks(l, n):
    # looping till length l
    lst = []
    for i in range(0, len(l), n):
        lst.append(l[i:i + n])
    return lst
def Ozs(V,r):
    if len(V) < r:
        V.append(1)
        for i in range(r - len(V)):
            V.append(0)
    assert len(V) == r
    return V
def list_xor(lst1,lst2):
    return [lst1[i]^lst2[i] for i in range(len(lst1))]
def int_to_bin(a,total_length):
    return list(map(int,format(a, f'0{total_length}b')))
def bits_list_to_hex_list(lst:list[int]) -> list[int]:
    assert len(lst) % 4 == 0
    new_list = []
    for i in range(len(lst)//4):
        a = int("".join(str(b) for b in lst[i*4:i*4+4]), 2)
        new_list.append(a)
    return new_list
def hex_list_to_bits_list(lst):
    new_list = []
    for i in lst:
        new_list += [int(digit) for digit in '{:04b}'.format(i)]
    return new_list


# Photon-Beetle Hash Function

In [4]:
def HASH(IV,D,r,c0):
    D_list = divide_chunks(D,r)
    D_list[-1] = Ozs(D_list[-1],r)
    for Di in D_list:
        Y_Z = hex_list_to_bits_list(PHOTON_256(bits_list_to_hex_list(IV)))
        Y,Z =Y_Z[:r],Y_Z[r:]
        W = list_xor(Y,Di)
        IV = W + Z
    IV = list_xor(IV , int_to_bin(c0,len(IV)))
    return IV
def TAG(T0,t=256):
    T_list = [T0]
    for i in range(t//128):
        T_list.append(hex_list_to_bits_list(PHOTON_256(bits_list_to_hex_list(T_list[-1]))))
    T = []
    for Ti in T_list:
        T += Ti[:128]
    return T
# THE MAIN HASH FUNCTION
def PHOTON_Beetle_Hash(M,r=128):
    M = copy.deepcopy(M)
    if len(M) == 0:
        IV = [0 for i in range(256)]
        T = TAG(list_xor(IV,int_to_bin(1,len(IV))),256)
        return T
    if len(M) <= 128:
        c0 = None
        if len(M) < 128 :
            c0 = 1
        else:
            c0 = 2    
        IV = Ozs(M,128) + [0 for i in range(128)]
        T = TAG(list_xor(IV,int_to_bin(c0,len(IV))),256)
        return T
    M1,M_ = M[:128] , M[128:]
    c0 = None
    if len(M_) % r == 0:
        c0 = 1
    else:
        c0 = 2
    IV = M1 + [0 for i in range(128)]
    IV = HASH(IV,M_,r,c0)
    T = TAG(IV,256)
    return T

# Testing

In [5]:
# GENERATING RANDOM LIST OF BITS
def generate_random_bit_list(l):
    lst = [ random.choice([0,1]) for i in range(l)]
    return lst
my_M = generate_random_bit_list(100)

In [6]:
hash_bits = PHOTON_Beetle_Hash(my_M,128)

In [7]:
print("Message :")
print("".join(map(str,my_M)))
print()
print("Hash :")
print("".join(map(str,hash_bits)))

Message :
0000111000011011010011111101000011100100000001001010101111011000110000011011110011011010110100010010

Hash :
000011100001101101001111110100001110010000000100101010111101100011000001101111001101101011010001001010000000000000000000000000001101101011101010011010000000100000001010001110011111001100001001100110011001111100100101000001100001001001110100000100111100111111110100110011110101101111110011101111011001011110011110101011011100011000101100111010000011101110110101110111000011111110011001


# - - - END - - -